In [1]:
import os
import tensorflow as tf #tf 2.0.0
import numpy as np
import pandas as pd
import pickle
from imutils import paths
import matplotlib.pyplot as plt
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import random
from sklearn.preprocessing import StandardScaler # import the scaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
import h5py
from tqdm import tqdm


# Image `Augmentation` for Increasing Dataset (Run Augmentation.py )

# ImageDataGenerator (in-place augmentation)
train_data_gen = ImageDataGenerator(rotation_range=50,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)
valid_data_gen = ImageDataGenerator(rotation_range=45,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)

test_data_gen = ImageDataGenerator(rescale=1./255)

dataset_dir = os.path.join('F:/GPU3/dataset11', "")

#dataset_dir = "E:/Progs/Restore/ManuscriptDataset/dataset11"

Batch_size =32
img_h = 299
img_w = 299
num_classes=4
n_epoch = 10

classes = ['crack', # 0
            'lackoffusion', # 1
            'porosity', # 2
            'slaginclusion', # 3
           ]

# Training
SEED = 42
#SEED = 1234
tf.random.set_seed(SEED) 

# initialize the data and labels
# x_train = []
# y_train = []
# x_test = []
# y_test = []


training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(299, 299),
                                               batch_size=Batch_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


# Validation
valid_dir = os.path.join(dataset_dir, 'testing')
valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                           target_size=(299, 299),
                                           batch_size=Batch_size, 
                                           classes=classes,
                                           class_mode='categorical',
                                           shuffle=False,
                                           seed=SEED)
# Test
test_dir = os.path.join(dataset_dir, 'testing')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(299, 299),
                                             batch_size=Batch_size,
                                             classes=classes,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

                                
print('Train gen =',train_gen.n)
train_gen.reset()
x_train, y_train = next(train_gen)
for i in tqdm(range(int((train_gen.n)/Batch_size))): #1st batch is already fetched before the for loop.
  img, label = next(train_gen)
  x_train = np.append(x_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(x_train.shape, y_train.shape)

valid_gen.reset()
x_test, y_test = next(valid_gen)
for i in tqdm(range(int((valid_gen.n)/Batch_size))): #1st batch is already fetched before the for loop.
  img, label = next(valid_gen)
  x_test = np.append(x_test, img, axis=0 )
  y_test = np.append(y_test, label, axis=0)
print(x_test.shape, y_test.shape)

#print('Train size = ',x_train.shape," ",y_train.shape)
#print('Test size = ',x_test.shape," ",y_test.shape)
# #Visualization data
# CLASS_NAMES = np.array(['devanagari','nandinagiri', 'newari','sharda'], dtype='<U10')


# def show_batch(image_batch, label_batch):
#   plt.figure(figsize=(25,20))
#   for n in range(8):
#       ax = plt.subplot(1,8,n+1)
#       plt.imshow(image_batch[n])
#       plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
#       plt.axis('off')
      
# image_batch, label_batch = next(train_gen)
# show_batch(image_batch, label_batch)


# ResNet152V2 Model


# The last 15 layers fine tune
# for layer in ResNet_model.layers[:-15]:
#     layer.trainable = False

# x = ResNet_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Flatten()(x)
# #x = Dense(units=2048, activation='relu')(x)
# #x = Dropout(0.25)(x)
# x = Dense(units=1024, activation='relu')(x)
# #x = Dropout(0.50)(x)
# output  = Dense(units=num_classes, activation='softmax')(x)
# model = Model(ResNet_model.input, output)
base_model = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

print("Successfully built the inception Model!")
print(model.summary())

loss = tf.keras.losses.CategoricalCrossentropy()
#loss = tf.keras.losses.KLDivergence()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])
print("Model Compilation completed!")

mc = ModelCheckpoint('F:/GPU3/Resultinception/best_modelresnet8.h5', monitor='val_accuracy', mode='max', save_best_only=True)
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

# lr_scheduler = LearningRateScheduler(lr_schedule)
#lrr = ReduceLROnPlateau(monitor='val_accuracy', 
                        # patience=3, 
                        # verbose=1, 
                        # factor=0.4, 
                        # min_lr=0.0001)


callbacks = [mc, es]

# model fit_generator
nb_train_samples = 1665
nb_validation_samples = 417

STEP_SIZE_TRAIN=nb_train_samples // Batch_size
STEP_SIZE_VALID=nb_validation_samples//Batch_size

# STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
# STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

import time 
tstart = time.time()
transfer_learning_history = model.fit_generator(generator=train_gen,
                   steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=valid_gen,
                   validation_steps=STEP_SIZE_VALID,
                   epochs=n_epoch,
                   #callbacks=[mc,es]
                   callbacks=callbacks,
                  
                    
)

tend = time.time()
elapsed = (tend - tstart)/60
print("Model trained Successfully : Took - {} mins!".format(elapsed))

# model evaluate with validation set
scores = model.evaluate(valid_gen,verbose=0)

#print(transfer_learning_history.history.keys())

print("Test Loss Value: %.2f%%" % (scores[0]*100))
print("Test Accuracy Value:  %.2f%%"  % (scores[1]*100))

STEP_SIZE_TEST=nb_validation_samples//Batch_size
test_gen.reset()
y_pred=model.predict(x_test,verbose=0)
y_pred=np.argmax(y_pred,axis=1)
y_test = np.argmax(y_test,axis = 1)

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, y_pred )
print('Accuracy: %.2f%%' % (accuracy*100))
# precision tp / (tp + fp)
precision = precision_score(y_test, y_pred ,average="macro" )
print('Precision: %.2f%%' % (precision*100))
# recall: tp / (tp + fn)
recall = recall_score(y_test, y_pred , average="macro")
print('Recall:%.2f%%' % (recall*100))
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, y_pred, average="macro")
print('F1 score: %.2f%%' % (f1*100))
report = classification_report(y_test, y_pred)
print(report)
# report.to_csv('E:/Progs/Restore/ManuscriptDataset/Resultcnnslabel/report.csv',sep = ',')

#print(classification_report(y_test, y_pred ))
# confusion matrix
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

# Plot non-normalized confusion matrix
titles_options = [
    
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_predictions(
        y_test,
        y_pred,
        display_labels=classes,
        cmap=plt.cm.Blues,
        normalize= normalize,
    )
    disp.ax_.set_title(title)
    print(title)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    print(disp.confusion_matrix)
    plt.savefig("F:/GPU3/Resultinception/confusion_matrix_resnet152_norm_8")

# Plot non-normalized confusion matrix
titles_options = [
    
    (" confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_predictions(
        y_test,
        y_pred,
        display_labels=classes,
        cmap=plt.cm.Blues,
        normalize= None,
    )
    disp.ax_.set_title(title)
    print(title)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    print(disp.confusion_matrix)
    plt.savefig("F:/GPU3/Resultinception/confusion_matrix_resnet152_8")
plt.show()

# save the model to disk
print("[INFO] serializing network...")
model.save("E:/Progs/Restore/modelresnet152_8  ", save_format="h5")
# save the single-label binarizer to disk
print("[INFO] serializing label binarizer...")
f = open("F:/GPU3/Resultinception/labelbinresnet152_8", "wb")
f.write(pickle.dumps(train_gen.class_mode))
f.close()

plt.style.use("ggplot")
plt.figure()
N=n_epoch

#plot the training loss and accuracy
acc = transfer_learning_history.history['accuracy']
val_acc = transfer_learning_history.history['val_accuracy']

loss = transfer_learning_history.history['loss']
val_loss = transfer_learning_history.history['val_loss']
plt.subplot(2, 1, 1)
plt.plot(np.arange(0,N), acc, label='Training Accuracy')
plt.plot(np.arange(0,N), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")

plt.subplot(2, 1, 2)
plt.plot(np.arange(0,N), loss, label='Training Loss')
plt.plot(np.arange(0,N), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel("Epoch #")
plt.ylabel("Loss")
#plt.tight_layout()
plt.tight_layout()
plt.savefig("F:/GPU3/Resultinception")
plt.show()




Found 1665 images belonging to 4 classes.
Found 417 images belonging to 4 classes.
Found 417 images belonging to 4 classes.
Train gen = 1665


 35%|████████████████████████████▍                                                     | 18/52 [00:31<00:54,  1.59s/it]C:\Users\DELL\anaconda3\lib\site-packages\PIL\Image.py:2896: DecompressionBombWarning: Image size (93006736 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [01:39<00:00,  1.92s/it]


(1665, 299, 299, 3) (1665, 4)


100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:09<00:00,  1.31it/s]


(417, 299, 299, 3) (417, 4)
Successfully built the inception Model!
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)               

C:\Users\DELL\AppData\Local\Temp\ipykernel_9948\2306601114.py:206: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  transfer_learning_history = model.fit_generator(generator=train_gen,


Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node 'model/conv2d_62/Conv2D' defined at (most recent call last):
    File "C:\Users\DELL\anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\DELL\anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\DELL\anaconda3\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\DELL\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\DELL\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\DELL\anaconda3\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    Fi